In [2]:
import flatbuffers
import sys
import argparse
import torch

sys.path.append("./converters")
sys.path.append("./generators")
sys.path.append("../")
from DataModel.Atom import Atom, AtomT
from DataModel.Structure import Structure, StructureT
from DataModel.Quaternion import Quaternion, QuaternionT
from DataModel.P import P, PT
from DataModel.AtomicModel import *
from DataModel.Parameters import Parameters, ParametersT

from DataModel.DataClass import DataClass
from DataModel.Datum import Datum, DatumT
from DataModel.Matrix import Matrix, MatrixT
from DataModel.Domain import Domain
from DataModel.DataType import DataType
from DataModel.DataSet import DataSet, DataSetT
from filegroup import *
from transform_generator import *
from ctf_generator import *
from image_generator import *
from dataloader import Dataloader as dl
import matplotlib.pyplot as plt

In [38]:
quats = gen_quat_torch(0)
print(len(quats))
pt = np.array([0, 1, 2], dtype=float)
pt = pt.reshape((3, 1))
print(pt.shape)
pt = torch.tensor(pt, dtype=torch.float64)
print(pt.shape)
qm = [quaternion_to_matrix(quat) for quat in quats]
# print(len(qm))
for i, q in enumerate(qm):
    # print(q.shape)
    res = torch.matmul(q, pt)
    print(res)

0
(3, 1)
torch.Size([3, 1])


In [41]:
st = StructureT.InitFromObj(
    Structure.GetRootAsStructure(dl.load_flatbuffer("struct_0.fbs"), 0)
)


def add_orientations(so, num_orientations):
    to_components = lambda q: {"a": q[0], "b": q[1], "c": q[2], "d": q[3]}
    quat_tensors = gen_quat_torch(num_orientations)
    quat_list = []
    for idx, qa in enumerate(quat_tensors):
        qt = QuaternionT()
        for k, v in to_components(qa).items():
            setattr(qt, k, v)
        quat_list.append(qt)
        # so = create_structureT_from_coords(numpy_array[i])
    so.orientations = quat_list
    return so


st = add_orientations(st, 1)
builder = flatbuffers.Builder(1024)
serialized_buffer = StructureT.Pack(st, builder)
builder.Finish(serialized_buffer)
sb = builder.Output()
f = open("oriented_struct_0.fbs", "wb")
f.write(sb)
f.close()